# ライブラリインポート

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup 
import csv

# 気候情報取得地域指定

In [2]:
place_codeA = [14, 34, 51, 44, 46, 61, 67, 63, 82]
place_codeB = [47412, 47590, 47636, 47662, 47670, 47759, 47765, 47770, 47807]
place_name = ["札幌", "仙台", "名古屋",  "東京", "横浜",  "京都", "広島", "神戸", "福岡"]   

# アウトプットファイルのパス指定

In [3]:
os.chdir("C://Users/dsuzuki/Desktop")

# ベースとなるURL指定

In [4]:
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=1&view=p1"

# 関数定義

取得するデータを文字型から浮動小数点型に変換する関数を定義

In [5]:
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

In [6]:
str(1)+str(1)

'11'

In [7]:
int(1)+int(1)

2

In [8]:
float(1)+float(1)

2.0

# スクレイピング実行

In [6]:
if __name__ == "__main__":
  #都市を網羅します
  for place in place_name:
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    All_list = [['年月日', '陸の平均気圧(hPa)', '海の平均気圧(hPa)', '降水量(mm)', '平均気温(℃)', '平均湿度(%)', '平均風速(m/s)', '日照時間(h)', '天気概況(昼)', '天気概況(夜)']]
    print(place)
    index = place_name.index(place)
    # for文で2018年~2019年までの11回。
    for year in range(2018,2020):
        #print(year)
        
        for month in range(1,13):
            r = requests.get(base_url%(place_codeA[index], place_codeB[index], year, month))
            r.encoding = r.apparent_encoding

        # 対象である表をスクレイピング。
            soup = BeautifulSoup(r.text)
            rows = soup.findAll('tr',class_='mtx') #タグ指定してclass名を指定するみたい。

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
            rows = rows[4:]

        # 1日〜最終日までの１行を網羅し、取得します。
            for row in rows:
                data = row.findAll('td')

          #１行の中には様々なデータがあるので全部取り出す。
          # ★ポイント
                rowData = [] #初期化
                rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
                rowData.append(str2float(data[1].string))
                rowData.append(str2float(data[2].string))
                rowData.append(str2float(data[3].string))
                rowData.append(str2float(data[6].string))
                rowData.append(str2float(data[9].string))
                rowData.append(str2float(data[11].string))
                rowData.append(str2float(data[16].string))
                rowData.append(data[19].string)
                rowData.append(data[20].string)

          #次の行にデータを追加
                All_list.append(rowData)
    #データフレーム化、表示            
    df_alllist = pd.DataFrame(All_list)
    print(df_alllist.head())
    #都市ごとにデータをファイルを新しく生成して書き出す。(csvファイル形式。名前は都市名)
    with open(place + '.csv', 'w') as file:
        writer = csv.writer(file, lineterminator='\n')
        writer.writerows(All_list)

札幌
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1005.7         1009      9.5      0.4       87        2.9   
2  2018/1/2       1002.7         1006        3      1.1       73        2.2   
3  2018/1/3       1006.3       1009.6      3.5     -0.7       71        2.3   
4  2018/1/4       1011.5       1014.8      4.5     -1.7       75          2   

         7         8           9  
0  日照時間(h)   天気概況(昼)     天気概況(夜)  
1      0.3  雪後みぞれ時々雨  みぞれ時々曇後一時雪  
2      2.5     雪後時々晴       曇後時々雪  
3      0.9      曇時々雪        雪一時曇  
4      0.1      雪時々曇        曇一時雪  
仙台
          0            1            2        3        4        5          6  \
0       年月日  陸の平均気圧(hPa)  海の平均気圧(hPa)  降水量(mm)  平均気温(℃)  平均湿度(%)  平均風速(m/s)   
1  2018/1/1       1006.1       1011.5        0      4.4       63        3.7   
2  2018/1/2       1007.5       1012.9        0        3       59        